In [ ]:
!pip install cloud-volume
!pip install caveclient

In [1]:
from caveclient import CAVEclient, auth
from caveclient import chunkedgraph as cg

In [2]:
import numpy as np
import pandas as pd

In [3]:
global_url = "https://global.brain-wire-test.org/"
local_url = "https://pcgv3local.brain-wire-test.org"
datastack = "fish1_full"
dataset = "fish1_v250915"
resolution = (16, 16, 30)

### Create CAVEclient Client

In [4]:
client = CAVEclient(datastack_name=datastack, server_address=global_url)

### Create ChunkedGraph Client

In [5]:
cggraph = cg.ChunkedGraphClient(server_address=local_url, table_name=dataset, auth_client=auth.AuthClient(token=client.auth.token))

### Get Segmentation Volume Data

In [6]:
cv = client.info.segmentation_cloudvolume()

/opt/conda/envs/pychunkedgraph/lib/python3.11/site-packages/google/rpc/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


### Lookup Segmentation Root ID by Coordinates

In [9]:
def get_latest_root_id(x, y, z):
    supervoxel_id = cv.download_point(pt=(x, y, z), 
                                      size=1, 
                                      agglomerate=False, 
                                      coord_resolution=resolution)
    supervoxel_id = np.int64(supervoxel_id[0, 0, 0, 0]) 
    print(f"supervoxel_id: {supervoxel_id}")
    root_id = cggraph.get_root_id(supervoxel_id)
    print(f"root_id: {root_id}")
    is_latest = cggraph.is_latest_roots([root_id])
    if not is_latest:
        latest_root_id = cggraph.get_latest_roots(root_id)
        print(f"latest_root_id: {latest_root_id}")
    else:
        latest_root_id = root_id
        print("root_id is the latest")
    
    return supervoxel_id, root_id, latest_root_id

In [10]:
get_latest_root_id(37880, 22443, 4595)

Decompressing: 100%|██████████| 1/1 [00:00<00:00, 389.84it/s]


supervoxel_id: 82407780375333845
root_id: 864691128652664051
root_id is the latest


(82407780375333845, 864691128652664051, 864691128652664051)